In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, minute, avg, countDistinct, rank, sum as _sum
from pyspark.sql.types import TimestampType
from pyspark.sql.window import Window

# Sample data
web_data = [
    Row(UserID=1, Page="Home", Timestamp="2024-04-10 10:00:00", Duration=35, Device="Mobile", Country="India"),
    Row(UserID=2, Page="Products", Timestamp="2024-04-10 10:02:00", Duration=120, Device="Desktop", Country="USA"),
    Row(UserID=3, Page="Cart", Timestamp="2024-04-10 10:05:00", Duration=45, Device="Tablet", Country="UK"),
    Row(UserID=1, Page="Checkout", Timestamp="2024-04-10 10:08:00", Duration=60, Device="Mobile", Country="India"),
    Row(UserID=4, Page="Home", Timestamp="2024-04-10 10:10:00", Duration=15, Device="Mobile", Country="Canada"),
    Row(UserID=2, Page="Contact", Timestamp="2024-04-10 10:15:00", Duration=25, Device="Desktop", Country="USA"),
    Row(UserID=5, Page="Products", Timestamp="2024-04-10 10:20:00", Duration=90, Device="Desktop", Country="India"),
]

df_web = spark.createDataFrame(web_data)



In [0]:
df_web.printSchema()


root
 |-- UserID: long (nullable = true)
 |-- Page: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Duration: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
df_web = df_web.withColumn("Timestamp", col("Timestamp").cast(TimestampType()))


In [0]:
df_web = df_web.withColumn("SessionMinute", minute("Timestamp"))
df_web.show(truncate=False)


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|Page    |Timestamp          |Duration|Device |Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|1     |Home    |2024-04-10 10:00:00|35      |Mobile |India  |0            |
|2     |Products|2024-04-10 10:02:00|120     |Desktop|USA    |2            |
|3     |Cart    |2024-04-10 10:05:00|45      |Tablet |UK     |5            |
|1     |Checkout|2024-04-10 10:08:00|60      |Mobile |India  |8            |
|4     |Home    |2024-04-10 10:10:00|15      |Mobile |Canada |10           |
|2     |Contact |2024-04-10 10:15:00|25      |Desktop|USA    |15           |
|5     |Products|2024-04-10 10:20:00|90      |Desktop|India  |20           |
+------+--------+-------------------+--------+-------+-------+-------------+



In [0]:
df_web.filter((col("Device") == "Mobile") & (col("Page") == "Checkout")).show()


+------+--------+-------------------+--------+------+-------+-------------+
|UserID|    Page|          Timestamp|Duration|Device|Country|SessionMinute|
+------+--------+-------------------+--------+------+-------+-------------+
|     1|Checkout|2024-04-10 10:08:00|      60|Mobile|  India|            8|
+------+--------+-------------------+--------+------+-------+-------------+



In [0]:
df_web.filter(col("Duration") > 60).show()


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|     2|Products|2024-04-10 10:02:00|     120|Desktop|    USA|            2|
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|
+------+--------+-------------------+--------+-------+-------+-------------+



In [0]:
df_web.filter((col("Country") == "India") & (col("Page") == "Products")).show()


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|
+------+--------+-------------------+--------+-------+-------+-------------+



In [0]:
df_web.groupBy("Device").agg(avg("Duration").alias("AvgDuration")).show()


+-------+------------------+
| Device|       AvgDuration|
+-------+------------------+
| Mobile|36.666666666666664|
| Tablet|              45.0|
|Desktop| 78.33333333333333|
+-------+------------------+



In [0]:
df_web.groupBy("Country").count().withColumnRenamed("count", "SessionCount").show()


+-------+------------+
|Country|SessionCount|
+-------+------------+
|  India|           3|
|    USA|           2|
|     UK|           1|
| Canada|           1|
+-------+------------+



In [0]:
from pyspark.sql.functions import desc

df_web.groupBy("Page").count().orderBy(desc("count")).limit(1).show()


+----+-----+
|Page|count|
+----+-----+
|Home|    2|
+----+-----+



In [0]:
window_spec = Window.partitionBy("UserID").orderBy("Timestamp")
df_web.withColumn("Rank", rank().over(window_spec)).show()


+------+--------+-------------------+--------+-------+-------+-------------+----+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|Rank|
+------+--------+-------------------+--------+-------+-------+-------------+----+
|     1|    Home|2024-04-10 10:00:00|      35| Mobile|  India|            0|   1|
|     1|Checkout|2024-04-10 10:08:00|      60| Mobile|  India|            8|   2|
|     2|Products|2024-04-10 10:02:00|     120|Desktop|    USA|            2|   1|
|     2| Contact|2024-04-10 10:15:00|      25|Desktop|    USA|           15|   2|
|     3|    Cart|2024-04-10 10:05:00|      45| Tablet|     UK|            5|   1|
|     4|    Home|2024-04-10 10:10:00|      15| Mobile| Canada|           10|   1|
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|   1|
+------+--------+-------------------+--------+-------+-------+-------------+----+



In [0]:
df_web.groupBy("UserID").agg(_sum("Duration").alias("TotalDuration")).show()


+------+-------------+
|UserID|TotalDuration|
+------+-------------+
|     1|           95|
|     3|           45|
|     2|          145|
|     4|           15|
|     5|           90|
+------+-------------+



In [0]:
df_web.createOrReplaceTempView("traffic_view")


In [0]:
spark.sql("""
    SELECT * FROM traffic_view
    ORDER BY Duration DESC
    LIMIT 2
""").show()


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|     2|Products|2024-04-10 10:02:00|     120|Desktop|    USA|            2|
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|
+------+--------+-------------------+--------+-------+-------+-------------+



In [0]:
spark.sql("""
    SELECT Page, COUNT(DISTINCT UserID) AS UniqueUsers
    FROM traffic_view
    GROUP BY Page
""").show()


+--------+-----------+
|    Page|UniqueUsers|
+--------+-----------+
|    Cart|          1|
|    Home|          2|
|Checkout|          1|
|Products|          2|
| Contact|          1|
+--------+-----------+



In [0]:
df_web.write.mode("overwrite").option("header", True).csv("/mnt/output/web_traffic_csv")


In [0]:
df_web.write.mode("overwrite").partitionBy("Country").parquet("/mnt/output/web_traffic_parquet")
